## Construction Sentiment analysis.

It will be analysed the sentiment analysis of "Construction" text, which was scrapped from Reddit, using the technique learned in Programming module. The text will be scrapped from Reddit in a j.son format, which is going to be transformed to a dataframe format to be later analysed, extracing information from text using lambda function and then performing a classification model.

An .env file is used to uload the API credentials.


In [1]:
!pip install python-dotenv

In [2]:
#Importing the libraries
from dotenv import load_dotenv
from os import getenv

In [3]:
#APP Info from Reddit
load_dotenv()

APP_NAME = getenv("APP_NAME")
APP_ID = getenv("APP_ID")
APP_SECRET = getenv("APP_SECRET")
USERNAME = getenv("REUSERNAME")# username is the account currently logged into windows
PASSWORD = getenv("PASSWORD")

In [4]:
USERNAME

In [5]:
USERNAME = getenv('REDDIT_USERNAME')

In [6]:
#loading the credentials from Reddit API
if APP_NAME and APP_ID and APP_SECRET and USERNAME and PASSWORD:
    print('Credentials loaded')
else:
    print("Error")

Credentials loaded


**OAuth2 Authentication FLow**

In [7]:
import requests

In [8]:
auth = requests.auth.HTTPBasicAuth(APP_ID, APP_SECRET)

data = {
    'grant_type': 'password',
    'username': USERNAME,
    'password': PASSWORD
}

headers = {'User-Agent': f'windows:{APP_NAME}:v1.0 (by /u/{USERNAME})'}

In [9]:
res = requests.post(
    'https://www.reddit.com/api/v1/access_token',
    auth=auth,
    data=data,
    headers=headers
)

In [10]:
res.status_code

200

In [11]:
res

<Response [200]>

In [12]:
TOKEN = res.json()['access_token']

In [13]:
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}

In [14]:
res = requests.get('https://oauth.reddit.com/r/construction/new', headers=headers)

In [15]:
res.status_code

200

In [16]:
#res = requests.get('https://oauth.reddit.com/r/construction materials/new',
                 #  headers=headers, params={'limit': '25'}) # it was previosly tested with a limit of 25

#the limit of comments were increased to 100
res = requests.get('https://oauth.reddit.com/r/construction/new',
                   headers=headers, params={'limit': '100', 'after': 't3_13f62c8'}) #'t3_13hefks'

In [17]:
print(res.json())

{'kind': 'Listing', 'data': {'after': 't3_13d9s7j', 'dist': 100, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'Construction', 'selftext': 'Whats behind this tile?  They want to drywall it. What should I do?.', 'author_fullname': 't2_2dr6ikhc', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Whats behind the backsplash', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/Construction', 'hidden': False, 'pwls': 6, 'link_flair_css_class': '', 'downs': 0, 'thumbnail_height': 140, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_13f5ofs', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 0.76, 'author_flair_background_color': None, 'ups': 2, 'total_awards_received': 0, 'media_embed': {}, 'thumbnail_width': 140, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, 'is_reddit_media_domain': True, 'is_meta':

In [18]:
#Printing the comments in j.son format
for post in res.json()['data']['children'][:]:
    print(post['data'])

{'approved_at_utc': None, 'subreddit': 'Construction', 'selftext': 'Whats behind this tile?  They want to drywall it. What should I do?.', 'author_fullname': 't2_2dr6ikhc', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Whats behind the backsplash', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/Construction', 'hidden': False, 'pwls': 6, 'link_flair_css_class': '', 'downs': 0, 'thumbnail_height': 140, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_13f5ofs', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 0.76, 'author_flair_background_color': None, 'ups': 2, 'total_awards_received': 0, 'media_embed': {}, 'thumbnail_width': 140, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, 'is_reddit_media_domain': True, 'is_meta': False, 'category': None, 'secure_media_embed': {}, 'link_flair_text': 'Question', 'can_mod_post': False, 'score': 2, 'approved_by': Non

In [19]:
#Importing the libraries to create dataframe
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

In [20]:
#Creating the dataframe wiht j.son data
df = pd.DataFrame()

for post in res.json()['data']['children']:
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    }, ignore_index=True)

In [21]:
#checking the id for the post
post['kind'] + '_' + post['data']['id']

't3_13d9s7j'

In [22]:
#post['data']['id']

In [23]:
#post['data'].keys()

In [24]:
#visualizing the 10 first rows of the dataframe
df.head(10)

,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2
1,Construction,A billion dollar view!,,0.78,22,0,22
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6
5,Construction,Road Paving Question,I see the same roads paved every five years an...,0.33,0,0,0
6,Construction,Anyone here specialize in fireproofing?,,1.00,1,0,1
7,Construction,This is what I do brothers.,,0.95,1054,0,1054
8,Construction,Boss: bring some wood in case there’s wood rep...,Christ help me,0.96,103,0,103
9,Construction,Tie off? I don't need that shit,,1.00,3,0,3


In [25]:
#checking the shape of the dataframe
df.shape

(100, 7)

In [26]:
#checking observation 1 for the variable "title"
df["title"][1]

'A billion dollar view!'

In [27]:
#extract the number of words for "title column"
# len of the list whenever I split my string. X the entire of the text and split it into individual words, 
# it will create the list and see the lenth of words
df["word_count"]=df["title"].apply(lambda x : len(str(x).split()))

In [28]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4
1,Construction,A billion dollar view!,,0.78,22,0,22,4
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16


In [29]:
#character of count column. It is going to count all the individual characters that I have in connection to this.
df["char_count"]=df["title"].str.len()

In [30]:
#checking the character of counts
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94


In [31]:
#Getting the average word lengh
# talking a loop inside the list. Talking each of the words and dividing it in the sum of words
def avg_word(sentence):
    words=sentence.split() # it will create a list of the individual words in that sentence
    return (sum(len(word) for word in words)/len(words))


In [32]:
# It is going to take each of the comments, it is going to run the function, each comment of title x value,  
#and pass into the funtion avarage word, average into each of the comments
df["avg_word"]= df["title"].apply(lambda x : avg_word(x))

In [33]:
#verifying the average word
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500


In [34]:
#import nltk
import nltk

In [35]:
#downloading "stopwords"
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
#defining stopwords for "English" language
from nltk.corpus import stopwords
stop = stopwords.words("english")

In [37]:
# Creating a new column "stopwords" as it goes into the single words, every single time, it does the enterely 
# of the stopwords library and have to do that for #every word, in every library. 
df['stopwords'] = df['title'].apply(lambda x: len([x for x in x.split() if x in stop]))
df.head() 

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000,1
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000,0
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727,3
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000,0
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500,0


In [38]:
# numeric characters within the text
df['numerics'] = df['title'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [39]:
#checking the numeric characters
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords,numerics
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000,1,0
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000,0,0
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727,3,0
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000,0,0
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500,0,0


In [40]:
#Creating a new column "title_mod" using lambda function
df["title_mod"]=df["title"].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [41]:
#visualizing the new column "title_mod"
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords,numerics,title_mod
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000,1,0,whats behind the backsplash
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000,0,0,a billion dollar view!
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727,3,0,most efficient way to do knockdown texture the...
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000,0,0,american plumbing question
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500,0,0,historic construction &amp; repair is a lot of...


In [42]:
#Removing any special character from text
df["title_mod"]=df["title_mod"].str.replace("[^\w\s]","")

In [43]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords,numerics,title_mod
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000,1,0,whats behind the backsplash
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000,0,0,a billion dollar view
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727,3,0,most efficient way to do knockdown texture the...
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000,0,0,american plumbing question
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500,0,0,historic construction amp repair is a lot of f...


In [44]:
#pip install textblob

In [45]:
#Importing textblow library
from textblob import TextBlob

In [46]:
# Extract the emotional value, the sentiment from this text
#first number is the sentimient, second is the number times this words is talked about
df["title_mod"][:5].apply(lambda x : TextBlob(x).sentiment)

0       (-0.4, 0.7)
1        (0.0, 0.0)
2        (0.5, 0.5)
3        (0.0, 0.0)
4    (0.2875, 0.55)
Name: title_mod, dtype: object

In [47]:
#creating a new column "sentiment"
#goingt through each of my comments and exgtracting the sentiment analysis stored into it
df["sentiment"]=df["title_mod"].apply(lambda x: TextBlob(x).sentiment[0])

In [48]:
#checking the sentiment of the text
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords,numerics,title_mod,sentiment
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000,1,0,whats behind the backsplash,-0.4000
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000,0,0,a billion dollar view,0.0000
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727,3,0,most efficient way to do knockdown texture the...,0.5000
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000,0,0,american plumbing question,0.0000
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500,0,0,historic construction amp repair is a lot of f...,0.2875


In [49]:
#checking the last five observations
df.tail()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords,numerics,title_mod,sentiment
95,Construction,What are the current issues in concrete constr...,I am doing coursework in trying to understand ...,0.33,0,0,0,8,53,5.750000,3,0,what are the current issues in concrete constr...,0.075
96,Construction,Would you be happy to get this from a homeowne...,"Title says most of it. New homeowner, moved in...",0.96,2017,0,2017,17,83,3.941176,8,0,would you be happy to get this from a homeowne...,0.800
97,Construction,"In your opinion, what would it take for you to...",I work for a company that does retail construc...,0.67,1,0,1,13,63,3.923077,7,0,in your opinion what would it take for you to ...,0.000
98,Construction,"In your opinion, what makes a good project man...",Hello everyone. I am currently a project engin...,1.00,2,0,2,9,51,4.777778,3,0,in your opinion what makes a good project manager,0.700
99,Construction,Tracking &amp; Receipting materials and equipment,Would love to get some input on how other comp...,1.00,1,0,1,6,49,7.333333,1,0,tracking amp receipting materials and equipment,0.000


In [50]:
#import numpy as np
#Applying lambda function to categorise the sentimente of the text either "negative" or positive.
df['sentiment_score'] = df['sentiment'].apply(lambda x: 'positive' if x>=0.1 else 'negative')

In [51]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,word_count,char_count,avg_word,stopwords,numerics,title_mod,sentiment,sentiment_score
0,Construction,Whats behind the backsplash,Whats behind this tile? They want to drywall ...,0.76,2,0,2,4,27,6.000000,1,0,whats behind the backsplash,-0.4000,negative
1,Construction,A billion dollar view!,,0.78,22,0,22,4,22,4.750000,0,0,a billion dollar view,0.0000,negative
2,Construction,Most efficient way to do knockdown texture the...,"what we did:\n\nspackle,\nscrape ceiling popco...",1.00,1,0,1,11,79,6.272727,3,0,most efficient way to do knockdown texture the...,0.5000,positive
3,Construction,American plumbing question,Why do Americans seem to love threaded steel f...,1.00,2,0,2,3,26,8.000000,0,0,american plumbing question,0.0000,negative
4,Construction,Historic Construction &amp; Repair Is A Lot Of...,,0.76,6,0,6,16,94,4.937500,0,0,historic construction amp repair is a lot of f...,0.2875,positive


In [52]:
#visualizing the array of sentiment score
df.sentiment_score.values

array(['negative', 'negative', 'positive', 'negative', 'positive',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'negative', 'negative', 'positive', 'positive',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'positive', 'positive', 'negative',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'negative', 'positive', 'negative', 'negative', 'negative',
       'negative', 'negative', 'positive', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'positive',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'positive', 'negative',
       'negative', 'positive', 'positive', 'positive', 'negative',
       'negative', 'positive', 'negative', 'positive', 'negati

In [53]:
#Importing pandas library to sub-set dataframe into new dataframe to extract "title" and "sentiment_score" columns
#to perform sentiment analysis
import pandas as pd
df_text = df[["title", "sentiment_score"]]
df_text.head()

,title,sentiment_score
0,Whats behind the backsplash,negative
1,A billion dollar view!,negative
2,Most efficient way to do knockdown texture the...,positive
3,American plumbing question,negative
4,Historic Construction &amp; Repair Is A Lot Of...,positive


In [54]:
#Selecting independent and target variable
X=df_text['title']
y=df_text['sentiment_score'] # target variable

In [55]:
# Display the value "X"
X

0                           Whats behind the backsplash
1                                A billion dollar view!
2     Most efficient way to do knockdown texture the...
3                            American plumbing question
4     Historic Construction &amp; Repair Is A Lot Of...
                            ...                        
95    What are the current issues in concrete constr...
96    Would you be happy to get this from a homeowne...
97    In your opinion, what would it take for you to...
98    In your opinion, what makes a good project man...
99    Tracking &amp; Receipting materials and equipment
Name: title, Length: 100, dtype: object

## Cleaning of Text Data

In [56]:
#loading stopwords library
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()

In [58]:
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()

In [59]:
#Importing re library 
import re

cleaned_data=[]

# For loop from first value to length(X), ^a-zA-Z means include small and capital case letters

for i in range(len(X)):
    df = re.sub('[^a-zA-Z]', ' ', X.iloc[i])
    df = df.lower().split()
    df = [stemmer.stem(word) for word in df if (word not in stop_words) and (word not in punct)]
    df = ' '.join(df)
    cleaned_data.append(df) 


In [60]:
# Display the cleaned_data
cleaned_data

['what behind backsplash',
 'billion dollar view',
 'effici way knockdown textur spray paint wall amp ceil',
 'american plumb question',
 'histor construct amp repair lot fun especi beauti place',
 'road pave question',
 'anyon special fireproof',
 'brother',
 'boss bring wood case wood replac behind side four wall',
 'tie need shit',
 'best way waterproof dincel structur wall',
 'day life beam sitter',
 'new outdoor job what fair price charg work',
 'much clearanc concret around water main',
 'career opinion top gc vs top gc',
 'softwar hanger load data visual',
 'today got poseidon kiss',
 'noth see',
 'timelaps construct perfect small wooden hous',
 'boy oh boy love engin around',
 'hypothet american venic',
 'true south florida job slow crawl',
 'help wall',
 'hotel construct superintend',
 'dyson dike',
 'appl juic anyon',
 'teamwork',
 'use old bathroom sink outsid ct',
 'job take long',
 'view work',
 'small remodel buis dad run manag',
 'updat fenway construct',
 'favorit tool'

In [61]:
print(y)

0     negative
1     negative
2     positive
3     negative
4     positive
        ...   
95    negative
96    positive
97    negative
98    positive
99    negative
Name: sentiment_score, Length: 100, dtype: object


In [62]:
# Collect all columns into dataframe named as sentiment_ordering
sentiment_ordering = ['negative', 'positive']

# store all values into column named as "y"
y = y.apply(lambda x: sentiment_ordering.index(x))

In [63]:
y.head()

0    0
1    0
2    1
3    0
4    1
Name: sentiment_score, dtype: int64

## Bag of Words using CountVectorizer

It be discussed the approach when the model will be trained for the task. The steps involved in performing sentiment analysis , which will be using the Bag of Words Vectorization method are as follows:

-Pre-Process the text of training data (Text pre-processing involves Normalization, Tokenization, Stopwords Removal, and Stemming/Lemmatization.)

-Create a Bag of Words for the pre-processed text data using the Count Vectorization and TF-IDF Vectorization approach.
Train a **MultinomialNB** classification model on the processed data for sentiment classification.

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate an object cv by calling a method named as CountVectorzer()
cv    = CountVectorizer(max_features = 3000, stop_words = ['construct', 'concrete'])

# Train the dataset by calling a fit_transform() method
X_fin = cv.fit_transform(cleaned_data).toarray()

# Display the rows and colums
X_fin.shape

(100, 348)

In [65]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Instantiate an object model by calling a method MultinomialNB()
model = MultinomialNB()

In [66]:
# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X_fin, y, test_size = 0.30)

In [67]:
# Train the model by calling a method fit()
model.fit(X_train,y_train)

MultinomialNB()

In [68]:
# Call predict() method
y_pred = model.predict(X_test)

In [69]:
# Caluclating the accuracy score of the model
from sklearn import metrics
#y_pred = model.predict(X_test)
accuracy_score = metrics.accuracy_score(y_pred, y_test)
print("Accuracuy Score: ",accuracy_score)

Accuracuy Score:  0.7


In [70]:
from sklearn.metrics import classification_report

# Instantiate a mthod named as Cla
cf = classification_report(y_test, y_pred)

# Display the values of an object cf
print(cf)

              precision    recall  f1-score   support

           0       0.81      0.77      0.79        22
           1       0.44      0.50      0.47         8

    accuracy                           0.70        30
   macro avg       0.63      0.64      0.63        30
weighted avg       0.71      0.70      0.71        30



**Discussion**: The trained classifier can be used to predict the sentiment, which produce an accuracy of **0.70.**

## Feature Generation using TF-IDF

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an object 'tf' by calling a method TfidfVectorizer()
tfidf = TfidfVectorizer(max_features = 3000, stop_words = ['construction', 'concrete'])

# Train the dataset by calling a method fit_tranform() 
X_tfidf = tfidf.fit_transform(cleaned_data).toarray()

In [72]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Instantiate an object model by calling a method MultinomialNB()
model_tdidf = MultinomialNB()

In [73]:
# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.30)

In [74]:
# Train the model by calling a method fit()
model_tdidf.fit(X_train, y_train)

MultinomialNB()

In [75]:
# Call predict() method
y_pred = model_tdidf.predict(X_test)

In [76]:
#Model evaluation
from sklearn.metrics import classification_report

# Instantiate a mthod named as Cla
cf = classification_report(y_test, y_pred)

# Display the values of an object cf
print(cf)

              precision    recall  f1-score   support

           0       0.80      1.00      0.89        24
           1       0.00      0.00      0.00         6

    accuracy                           0.80        30
   macro avg       0.40      0.50      0.44        30
weighted avg       0.64      0.80      0.71        30



**Discussion**: The trained classifier using TfidfVectorizercan produced an accuracy of **0.80.** Overall, both methods can be used to analyse the sentiment of construction text.

 ## Analysis of text "Construction" using VADER  rule-based sentiment analyzer.

In [77]:
#pip install vaderSentiment

In [78]:
# Load thelibraries
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Create and initialise an object
sentiment = SentimentIntensityAnalyzer()
text_1 = "Starting as an asphalt laborer tomorrow."
text_2 =  "Nooo - my scrolling thumb :("

sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)

print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)

Sentiment of text 1: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentiment of text 2: {'neg': 0.367, 'neu': 0.633, 'pos': 0.0, 'compound': -0.4404}


**Discusion of the results:**

- a positive sentiment, compound ≥ 0.05.
- a negative sentiment, compound ≤ -0.05.
- a neutral sentiment, the compound is between ]-0.05, 0.05[

**Text 1**: The previous result shows that the there is no negative information (neg=0). There is some neutral tone (neu=1.0), no positive tone (pos=0). There is a positive tones (0.318). 

**Text 2**: Result shows that the there is negative information (neg=0.367). There is some neutral tone (neu=0.633), no positive tone (pos=0). However, the overall sentiment is negative, because compound < 0.05.

In [79]:
# Install and import nltk
!pip install nltk
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [81]:
# Import the lexicon 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

# Example
sentence = "what best way get start residential construct goal build hours"
#sentence = "A bit of a frustrated rant here"
print(sent_analyzer.polarity_scores(sentence))

{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.6369}


**Discusion of the results:**

- a positive sentiment, compound ≥ 0.05.
- a negative sentiment, compound ≤ -0.05.
- a neutral sentiment, the compound is between ]-0.05, 0.05[

The previous result shows that the there is no negative information (neg=0). There is some neutral and postive tone (neu=0.682 and pos=0.318). There is a positive tones (0.318). However, the overall sentiment is positive because compound > 0.05.

**References**

<p>https://www.analyticsvidhya.com/blog/2022/07/sentiment-analysis-using-python/</p>